<a href="https://colab.research.google.com/github/QaziSaim/CASE-STUDIES/blob/main/PPO_Proximal_Policy_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import numpy as np
import gym

In [2]:
# --- Hyperparameters ---
gamma = 0.99
clip_ratio = 0.2
lr_actor = 0.0003
lr_critic = 0.001
lam = 0.95
epochs = 10
batch_size = 64
train_policy_iters = 80
train_value_iters = 80
steps_per_epoch = 4000

In [3]:
env = gym.make('CartPole-v1')

/usr/local/lib/python3.12/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.12/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [18]:
obs_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

In [7]:
print(env.observation_space)
print(env.observation_space.shape)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
(4,)


In [5]:
obs_dim

4

In [9]:
env.action_space.n

2

In [10]:
env.action_space

Discrete(2)

In [12]:
env.reward_range

(-inf, inf)

In [15]:
env.render_mode

In [17]:
print(env.spec)

EnvSpec(id='CartPole-v1', entry_point='gym.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=475.0, nondeterministic=False, max_episode_steps=500, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={}, namespace=None, name='CartPole', version=1)


In [21]:
def get_actor():
  inputs = keras.Input(shape=(obs_dim,))
  x = layers.Dense(64, activation='tanh')(inputs)
  x = layers.Dense(64, activation='tanh')(x)
  logits = layers.Dense(n_actions,activation=None)(x)
  model = Model(inputs=inputs,outputs=logits)
  return model

In [22]:
def get_critic():
  inputs=keras.Input(shape=(obs_dim,))
  x = layers.Dense(64,activation='tanh')(inputs)
  x = layers.Dense(64,activation='tanh')(x)
  logits  = layers.Dense(1)(x)
  model = Model(inputs=inputs,outputs=logits)
  return model

In [29]:
actor=get_critic()
critic = get_actor()

In [35]:
print(actor.summary())
print('\n')
print(critic.summary())

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,545 (17.75 KB)

 Trainable params: 4,545 (17.75 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,610 (18.01 KB)

 Trainable params: 4,610 (18.01 KB)

 Non-trainable params: 0 (0.00 B)

None


In [36]:
actor_optimizer = keras.optimizers.Adam(learning_rate=lr_actor)
cirtic_optimizer = keras.optimizers.Adam(learning_rate=lr_critic)